In [7]:
import mplhep as hep
import matplotlib.pyplot as plt
from wremnants.datasets.datagroups import datagroups2016
from utilities import boostHistHelpers as hh,common
from wremnants import plot_tools,syst_tools
from wremnants import histselections as sel
import hist
import narf
import numpy as np
import pickle,lz4.frame

2023-01-12 17:00:28.113253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 17:00:28.727206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs
2023-01-12 17:00:28.727490: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-12 17:00:28.849869: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-12 17:00:30.458151: W

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
groups = datagroups2016("../mw_with_mu_eta_pt_nnpdf31.pkl.lz4")


In [4]:
groups.results["WplusmunuPostVFP"]["output"].keys()

dict_keys(['nominal', 'qcdJetPt45', 'effStatTnP', 'effStatTnP_tracking', 'effStatTnP_reco', 'effSystTnP', 'muonL1PrefireStat', 'muonL1PrefireSyst', 'ecalL1Prefire', 'luminosity', 'qcdScale', 'qcdScaleByHelicity', 'pdfNNPDF31', 'alphaS002pdfNNPDF31', 'massWeight', 'muonScaleSyst', 'muonScaleSyst_responseWeights'])

In [5]:
# This is for reading groups
groups.loadHistsForDatagroups("nominal", syst="", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="muonScaleSyst", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="muonScaleSyst_responseWeights", procsToRead=["Wmunu"])
histInfo = groups.getDatagroups()

In [6]:
# This is for reading groups
groups.loadHistsForDatagroups("nominal", syst="", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="qcdScaleByHelicity", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="pdfNNPDF31", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="pdfMMHT", procsToRead=["Wmunu"])
groups.loadHistsForDatagroups("", syst="pdfCT18", procsToRead=["Wmunu"])
histInfo = groups.getDatagroups()

ValueError: Did not find systematic pdfMMHT for any processes!

In [ ]:
histInfo["Wmunu"]["pdfCT18"]

In [ ]:
hh.divideHists(histInfo["Wmunu"]["pdfMMHT"][{"tensor_axis_0" : 0}].project("eta"), histInfo["Wmunu"]["nominal"].project("eta")).plot(label="MMHT14")
hh.divideHists(histInfo["Wmunu"]["pdfCT18"][{"tensor_axis_0" : 0}].project("eta"), histInfo["Wmunu"]["nominal"].project("eta")).plot(label="CT18")
plt.legend()
plt.ylim([0.95, 1.05])


In [ ]:
hh.divideHists(histInfo["Wmunu"]["pdfMMHT"][{"tensor_axis_0" : 0}].project("pt"), histInfo["Wmunu"]["nominal"].project("pt")).plot(label="MMHT14")
hh.divideHists(histInfo["Wmunu"]["pdfCT18"][{"tensor_axis_0" : 0}].project("pt"), histInfo["Wmunu"]["nominal"].project("pt")).plot(label="CT18")
plt.legend()
plt.ylim([0.95, 1.05])


In [ ]:
hscaleUp = procDict["Wmunu"]["muonScaleSyst"][{"downUpVar" : 1, "scaleEtaSlice" : 1.j}]
hscaleDown = procDict["Wmunu"]["muonScaleSyst"][{"downUpVar" : 0, "scaleEtaSlice" : 1.j}]

In [ ]:
hnom = procDict["Wmunu"]["nominal"]
# Can also read specific samples by accessing the original 
hnomminus = groups.results["WminusmunuPostVFP"]["output"]["nominal"]

In [ ]:
hscaleUp.project("pt").plot()

In [ ]:
hh.divideHists(hscaleUp.project("pt"), hnom.project("pt")).plot()
hh.divideHists(hscaleDown.project("pt"), hnom.project("pt")).plot()
plt.ylim([0.995, 1.005])

In [ ]:
combine = datagroups2016("../temp/WMassCombineInput.root")

In [ ]:
procs = combine.datagroupsForHist("x_Wmunu_plus", label="nominal")
procs = combine.datagroupsForHist("x_Wmunu_CMS_scale_m_ieta0Up_plus", label="scaleEta0Up")
procs = combine.datagroupsForHist("x_Wmunu_CMS_scale_m_ieta0Down_plus", label="scaleEta0Down")
procs = combine.datagroupsForHist("x_Wmunu_pdf1NNPDF31Up_plus", label="pdf1Up")
procs = combine.datagroupsForHist("x_Wmunu_pdf1NNPDF31Down_plus", label="pdf1Down")

In [ ]:
hh.divideHists(procs["Wmunu"]["scaleEta0Up"], procs["Wmunu"]["nominal"]).plot()
hh.divideHists(procs["Wmunu"]["scaleEta0Down"], procs["Wmunu"]["nominal"]).plot()
#plt.ylim([0.99, 1.01])

In [ ]:
hmirror = hh.mirrorHist(hnom, procs["Wmunu"]["pdf1Up"])

In [ ]:
hh.divideHists(procs["Wmunu"]["pdf1Up"].project("pt"),procs["Wmunu"]["nominal"].project("pt")).plot()
hh.divideHists(procs["Wmunu"]["pdf1Down"].project("pt"),procs["Wmunu"]["nominal"].project("pt")).plot()
plt.ylim([0.99, 1.01])

In [ ]:
hists = [sel.unrolledHist(h) for h in [procs["Wmunu"]["nominal"], procs["Wmunu"]["scaleEta0Up"], procs["Wmunu"]["scaleEta0Down"]]]
#hists = [h.project("pt") for h in [procs["Wmunu"]["nominal"], procs["Wmunu"]["pdf1Up"], procs["Wmunu"]["pdf1Down"]]]
colors = ["black", "red", "pink"]
labels = ["nominal", "scaleEta0Up", "scaleEta0Down"]

fig = plotTools.makePlotsWithRatioToRef(hists, labels, colors, xlabel=r"($\eta$, p$_{T}$) bin", scale=1e5,rrange=[0.998, 1.002])


In [ ]:
hists = [sel.unrolledHist(h) for h in [procs["Wmunu"]["nominal"], procs["Wmunu"]["pdf1Up"], procs["Wmunu"]["pdf1Down"]]]
colors = ["black", "red", "pink"]
labels = ["nominal", "pdf1Up", "pdf1Down"]

fig = plotTools.makePlotsWithRatioToRef(hists, labels, colors, xlabel=r"($\eta$, p$_{T}$) bin", scale=1e5,rrange=[0.995, 1.005])


In [ ]:
histInfo = groups.getDatagroups()
groups.loadHistsForDatagroups("nominal", syst="", selectSignal=True, label="nominal", procsToRead=["Wmunu"])

In [ ]:
groups.loadHistsForDatagroups("", syst="qcdScaleByHelicity", selectSignal=True, label="qcdScaleByHelicity", procsToRead=["Wmunu"], forceNonzero=False)

In [ ]:
histInfo["Wmunu"]["qcdScaleByHelicity"]

In [ ]:
histInfo["Wmunu"]["qcdScaleByHelicitySum"] = syst_tools.scale_helicity_hist_to_variations(histInfo["Wmunu"]["qcdScaleByHelicity"])

In [ ]:
s = hist.tag.Slicer()
hVarNom = histInfo["Wmunu"]["qcdScaleByHelicity"][{"ptVgen" : s[::hist.sum], "chargeVgen" : s[::hist.sum], "helicity" : s[::hist.sum], "muRfact" : s[1.j], "muFfact" : s[1.j] }]


In [ ]:
selections = [{"muRfact" : hist.loc(j), "muFfact" : hist.loc(j), "helicity" : i, "ptVgen" : 0, "chargeVgen" : -1j} for i in range(2) for j in [0.5,1.,2.]]
names = [f"muR{j}_muF{j}_hel_sigma{i}_ptVgen0_minus" for i in range(2) for j in [0.5,1.,2.]]

In [ ]:
hists = [sel.unrolledHist(histInfo["Wmunu"]["nominal"])]
hists.extend([sel.unrolledHist(histInfo["Wmunu"]["qcdScaleByHelicitySum"][s]) for s in selections])
labels = ["nominal"]+names
colors = ["black", "blue", "purple", "green", "pink", "orange", "lightblue"]

In [ ]:
hists = [histInfo["Wmunu"]["nominal"].project("pt")]
hists.extend([histInfo["Wmunu"]["qcdScaleByHelicitySum"][s].project("pt") for s in selections])

In [ ]:
fig = plot_tools.makePlotWithRatioToRef(hists, labels, colors, xlabel=r"p$_{T}$ (GeV)", ymax=3e7,rrange=[0.99, 1.01])

In [ ]:
selections = [{"muRfact" : 1.j, "muFfact" : 1.j, "helicity" : i} for i in range(9)]
#histInfo["Wmunu"]["qcdScaleByHelicity"] = syst_tools.scale_helicity_hist_to_variations(histInfo["Wmunu"]["qcdScaleByHelicity"], sum_ptV=True)
hists = [sel.unrolledHist(histInfo["Wmunu"]["nominal"])]
hists.extend([sel.unrolledHist(histInfo["Wmunu"]["qcdScaleByHelicity"][s]) for s in selections])
labels = ["nominal"]+[f"$sigma_{i-1}$*angular" if i != 0 else "$\sigma_\mathrm{UL}$" for i in range(9)]
colors = ["black", "blue", "purple", "green", "pink", "orange", "lightblue", "red", "lightgreen", "yellow"]
print(len(hists), len(labels), len(colors))
fig = plot_tools.makePlotWithRatioToRef(hists, labels, colors, xlabel=r"($\eta_{\ell}$, p$_{T}$) bin", ymax=8e6,rrange=[-0.2, 1.5])
fig.get_axes()[0].set_ylim([-1e4, 1.8e5])

In [ ]:
angular = [
    r"(1+\cos^{2}\theta)",
    r"(1-3\cos^{2}\theta)",
    r"\sin2\theta\cos\phi",
    r"\sin^{2}\theta",
    r"\sin\theta\cos\phi",
    r"\cos\phi",
    r"\sin^{2}\theta\sin2\theta",
    r"\sin2\theta\sin\phi",
    r"\sin\theta\sin\phi",
]

In [ ]:
selections = [{"muRfact" : 1.j, "muFfact" : 1.j, "helicity" : i} for i in range(9)]
#histInfo["Wmunu"]["qcdScaleByHelicity"] = syst_tools.scale_helicity_hist_to_variations(histInfo["Wmunu"]["qcdScaleByHelicity"], sum_ptV=True)
hists = [sel.unrolledHist(histInfo["Wmunu"]["nominal"])]
hists.extend([sel.unrolledHist(histInfo["Wmunu"]["qcdScaleByHelicity"][s]) for s in selections])
labels = ["nominal"]+["$\sigma_{%i}%s$" % (i-1, angular[i]) for i in range(0,9)]
colors = ["black", "blue", "purple", "green", "pink", "orange", "lightblue", "red", "lightgreen", "yellow"]
print(len(hists), len(labels), len(colors))
fig = plot_tools.makePlotWithRatioToRef(hists, labels, colors, xlabel=r"($\eta_{\ell}$, p$_{T}$) bin", ymax=8e6,rrange=[0.9, 1.1], rlabel="$x/\sigma_{total}$")
fig.get_axes()[0].set_ylim([-1e4, 1.8e5])

In [ ]:
selections = [{"muRfact" : 1.j, "muFfact" : 1.j, "helicity" : i} for i in range(9)]
hists = [sel.unrolledHist(histInfo["Wmunu"]["nominal"])]
hists.extend([sel.unrolledHist(histInfo["Wmunu"]["qcdScaleByHelicity"][s]) for s in selections])
labels = ["nominal"]+["$\sigma_{%i}%s$" % (i-1, angular[i]) for i in range(0,9)]
colors = ["black", "blue", "purple", "green", "pink", "orange", "lightblue", "red", "lightgreen", "yellow"]
print(len(hists), len(labels), len(colors))
fig = plot_tools.makePlotWithRatioToRef(hists, labels, colors, xlabel=r"($\eta_{\ell}$, p$_{T}$) bin", ymax=8e6,rrange=[-0.05, 0.1])
fig.get_axes()[0].set_ylim([-1e4, 1.8e5])

In [ ]:
histInfo["Wmunu"]["qcdScaleByHelicity"]

In [ ]:
histInfo["Wmunu"]["nominal"].project("pt").plot()
histInfo["Wmunu"]["qcdScaleByHelicity"][{"muRfact" : 1.j, "muFfact" : 1.j}].project("pt").plot()

In [ ]:
print(groups.results["WminusmunuPostVFP"]["output"]["nominal"].sum(flow=True))
print(groups.results["WminusmunuPostVFP"]["output"]["qcdScale"][{"muRfact" : 1.j, "muFfact" : 1.j}].sum(flow=True))


In [ ]:
groups.results["WminusmunuPostVFP"]["output"]["nominal"].project("pt").plot(label="nominal")
groups.results["WminusmunuPostVFP"]["output"]["qcdScaleByHelicity"][{"muRfact" : 1.j, "muFfact" : 1.j}].project("pt").plot()
plt.legend()

In [ ]:
groups.results["WplusmunuPostVFP"]["output"]["nominal"].project("pt").plot(label="nominal")
groups.results["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"][{"muRfact" : 1.j, "muFfact" : 1.j}].project("pt").plot()
plt.legend()

In [ ]:
groups.results["WminusmunuPostVFP"]["output"]["qcdScaleByHelicity"]

In [ ]:
#groups.results["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"]
groups.results["WminusmunuPostVFP"]["output"]["qcdScaleByHelicity"]

In [ ]:
orig = groups.results["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"].copy()
orig

In [ ]:
orig_sum = orig +  groups.results["WminusmunuPostVFP"]["output"]["qcdScaleByHelicity"]

In [ ]:
groups.loadHistsForDatagroups("nominal", syst="qcdScaleByHelicity", procsToRead=["Wmunu"], forceNonzero=False, selectSignal=False,
    preOpMap={x : syst_tools.scale_helicity_hist_to_variations for x in common.vprocs},
    preOpArgs={"sum_axis" : [ "helicity", "ptVgen",]})

In [ ]:
groups.loadHistsForDatagroups("nominal", syst="qcdScaleByHelicity", procsToRead=["Zmumu"], forceNonzero=False, selectSignal=False,               
    preOpMap={x : syst_tools.scale_helicity_hist_to_variations for x in common.vprocs},
    preOpArgs={"sum_axis" : ["ptVgen",]})

In [ ]:
groups.loadHistsForDatagroups("nominal", syst="", procsToRead=["Fake",], selectSignal=True, forceNonzero=False)

In [ ]:
groups.loadHistsForDatagroups("nominal", syst="qcdScale", procsToRead=["Fake",], forceNonzero=False, selectSignal=True,
    preOpMap={x : syst_tools.scale_helicity_hist_to_variations for x in common.vprocs},
                             )#preOpArgs={"sum_axis" : ["ptVgen"]})

In [ ]:
res = groups.getDatagroups()

In [ ]:
res["Fake"]["qcdScale"][{"muRfact" : 1.j, "muFfact" : 1.j, "ptVgen" : 10, "chargeVgen" : -1.j}]

In [ ]:
res["Fake"]["nominal"]

In [ ]:
hh.divideHists(res["Fake"]["qcdScale"][{"muRfact" : 0.5j, "muFfact" : 0.5j, "ptVgen" :1, "chargeVgen" : 1.j}].project("pt"),
    res["Fake"]["nominal"].project("pt")).plot()
hh.divideHists(res["Fake"]["qcdScale"][{"muRfact" : 2j, "muFfact" : 2j, "ptVgen" :1, "chargeVgen" : 1.j}].project("pt"),
    res["Fake"]["nominal"].project("pt")).plot()
plt.ylim([0.9, 1.1])

In [ ]:
res["Fake"]["qcdScaleByHelicity"][{"muRfact" : 2j, "muFfact" : 2j}].project("pt").plot()
res["Fake"]["nominal"].project("pt").plot()



In [ ]:
read = pickle.load(lz4.frame.open("../mw_with_mu_eta_pt_nnpdf31.pkl.lz4", "rb"))

In [ ]:
print(res["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"][{"helicity" : s[::hist.sum], "muRfact" : 1.j, "muFfact" :1.j}].sum())
print(res["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"][{"helicity" : -1.j, "muRfact" : -1.j, "muFfact" :1.j,}].sum())

In [ ]:
groups.results["WplusmunuPostVFP"]["output"]["qcdScaleByHelicity"][{"chargeVgen" : s[::hist.sum], "ptVgen" : s[::hist.sum], "helicity" : s[::hist.sum], "muRfact" : -1.j, "muFfact" :1.j,}]

In [ ]:
groups.results["WplusmunuPostVFP"]["output"]["nominal"]

In [ ]:
res["Wmunu"]["qcdScaleByHelicity"][{"chargeVgen" : 1.j, "muRfact" : 1j, "muFfact" : 1j, "helicity" : 4.j, "ptVgen" : 1}]

In [ ]:
for i in range(8):
    hh.divideHists(res["Wmunu"]["qcdScaleByHelicity"][{"chargeVgen" : 1.j, "muRfact" : 2j, "muFfact" : 2j, "helicity" : i, }].project("pt"),
    groups.results["WplusmunuPostVFP"]["output"]["nominal"].project("pt")).plot()
    hh.divideHists(res["Wmunu"]["qcdScaleByHelicity"][{"chargeVgen" : 1.j, "muRfact" : 0.5j, "muFfact" : 0.5j, "helicity" : i, }].project("pt"),
    groups.results["WplusmunuPostVFP"]["output"]["nominal"].project("pt")).plot()
plt.ylim([0.96, 1.04])

In [ ]:
print(res["Fake"]["nominal"][{"charge" : 1.j}].sum())
print(res["Fake"]["nominal"][{"charge" : -1.j}].sum())

In [ ]:
res["Fake"]["qcdScale"].axes["ptVgen"].size

In [ ]:
import uproot
combf = uproot.open("../test/WMassCombineInput.root")

In [ ]:
print(combf["x_Fake_plus"].to_hist().sum())
print(combf["x_Fake_minus"].to_hist().sum())

In [ ]:
refm = combf["x_Fake_minus"].to_hist().sum().value
refp = combf["x_Fake_plus"].to_hist().sum().value

In [ ]:
res["Fake"]["qcdScale"].axes["ptVgen"].size

In [ ]:
for c in ["minus", "plus"]:
    for i in range(0,int(res["Fake"]["qcdScale"].axes["ptVgen"].size/2)):
        print("Q=", c, "bin", i, combf[f"x_Fake_QCDscale_genV{c}genPtV{i}muRmuFDown_plus"].to_hist().sum().value/refp)

In [25]:
with lz4.frame.open("/scratch/kelong/Analysis/mz_wlike_with_mu_eta_pt_RawPFMET_scetlibCorr_nnpdf31.pkl.lz4") as f:
    res = pickle.load(f)

In [27]:
res["ZmumuPostVFP"]["output"]["massWeight"]

Hist(
  Regular(60, 60, 120, name='mll'),
  Regular(25, -2.5, 2.5, name='yll'),
  Variable(array([  0.  ,   2.  ,   3.  ,   4.  ,   4.75,   5.5 ,   6.5 ,   8.  ,
         9.  ,  10.  ,  12.  ,  14.  ,  16.  ,  18.  ,  20.  ,  23.  ,
        27.  ,  32.  ,  40.  ,  55.  , 100.  , 150.  ]), name='ptll'),
  Regular(2, -2, 2, underflow=False, overflow=False, name='charge'),
  Integer(0, 21, underflow=False, overflow=False, name='tensor_axis_0'),
  storage=Weight()) # Sum: WeightedSum(value=4.04745e+08, variance=4.70231e+08) (WeightedSum(value=4.06904e+08, variance=4.7249e+08) with flow)